In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [28]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

<h1>Load and Prep dataset</h1>

In [29]:
data = pd.read_csv('/Users/Curran/gatech/Summer 2023/CS 4641/Project/data/dataset.csv')
data.head()

,Symbol,Date,Open,High,Low,Close,Volume
0,A,2009-03-02 05:00:00,9.73,9.80,9.02,9.07,5994808
1,A,2009-03-03 05:00:00,9.22,9.34,8.88,9.07,7057671
2,A,2009-03-04 05:00:00,9.24,9.70,9.11,9.52,5427775
3,A,2009-03-05 05:00:00,9.28,9.46,8.95,8.97,9678690
4,A,2009-03-06 05:00:00,8.91,9.36,8.60,9.05,6800559


In [30]:
# Copy the data and sort by symbol and date
df = data.copy().sort_values(by=["Symbol", "Date"])
df["Date"] = pd.to_datetime(df["Date"])
df.set_index("Date", drop=True, inplace=True)
df.head()

,Symbol,Open,High,Low,Close,Volume
Date,,,,,,
2009-03-02 05:00:00,A,9.73,9.80,9.02,9.07,5994808
2009-03-03 05:00:00,A,9.22,9.34,8.88,9.07,7057671
2009-03-04 05:00:00,A,9.24,9.70,9.11,9.52,5427775
2009-03-05 05:00:00,A,9.28,9.46,8.95,8.97,9678690
2009-03-06 05:00:00,A,8.91,9.36,8.60,9.05,6800559


In [31]:
# Group by symbol and describe the data
stats_by_stock:pd.DataFrame = df.groupby("Symbol").describe()
stats_by_stock.head()
# Uncomment to save the stats to a file
# stats_by_stock.to_excel("./stats_by_stock.xlsx")

KeyboardInterrupt: 

In [ ]:
# Select valid symbols
upper_bound = stats_by_stock["Open"]["max"] < 500
lower_bound = stats_by_stock["Open"]["min"] > 1
outliers_removed = stats_by_stock[upper_bound & lower_bound]
valid_symbols = outliers_removed.index.to_list()

# Remove outliers from the data
mask = df["Symbol"].isin(valid_symbols)
df = df[mask]
print(df.shape)
df.head()

(4784832, 6)


,Symbol,Open,High,Low,Close,Volume
Date,,,,,,
2009-03-02 05:00:00,A,9.73,9.80,9.02,9.07,5994808
2009-03-03 05:00:00,A,9.22,9.34,8.88,9.07,7057671
2009-03-04 05:00:00,A,9.24,9.70,9.11,9.52,5427775
2009-03-05 05:00:00,A,9.28,9.46,8.95,8.97,9678690
2009-03-06 05:00:00,A,8.91,9.36,8.60,9.05,6800559


<h1>Feature Engineering</h1>

In [ ]:
df.sort_values(by=['Symbol', 'Date'], inplace=True)

df.reset_index(drop=False, inplace=True)
df.head()
df_weekly = pd.DataFrame()

for symbol, stock_data in df.groupby('Symbol'):
    weekly_stock_data = stock_data.iloc[::5]
    df_weekly = df_weekly.append(weekly_stock_data)

df_weekly.reset_index(drop=True, inplace=True)

In [ ]:
df_weekly.tail()

,Date,Symbol,Open,High,Low,Close,Volume
957307,2020-01-27 05:00:00,ZUMZ,31.28,32.53,30.97,31.93,335722
957308,2020-02-03 05:00:00,ZUMZ,31.49,32.80,31.18,32.59,318451
957309,2020-02-10 05:00:00,ZUMZ,31.86,32.44,31.42,32.12,162419
957310,2020-02-18 05:00:00,ZUMZ,32.51,33.17,32.23,32.39,262345
957311,2020-02-25 05:00:00,ZUMZ,30.75,30.75,29.44,29.53,361695


In [ ]:
df_weekly.head()

,Date,Symbol,Open,High,Low,Close,Volume
0,2009-03-02 05:00:00,A,9.73,9.80,9.02,9.07,5994808
1,2009-03-09 04:00:00,A,8.94,9.28,8.83,8.90,6017432
2,2009-03-16 04:00:00,A,10.11,10.19,9.84,9.84,3028316
3,2009-03-23 04:00:00,A,10.73,11.37,10.73,11.37,6473243
4,2009-03-30 04:00:00,A,11.32,11.34,10.83,11.01,5550662


In [ ]:
#Introduce a common financial statistic, 10 or 30 day momentum
#shift pulls the close for a stock 10 days prior

df_weekly['log_volume'] = df_weekly['Volume'].apply(np.log)

df_weekly['Momentum_2Wk'] = df_weekly['Close'] - df_weekly['Close'].shift(2)

df_weekly['Momentum_2Wk'] = df_weekly['Close'] - df_weekly['Close'].shift(5)

df_weekly = df_weekly.groupby('Symbol').apply(lambda x: x.iloc[5:])

df_weekly.set_index(['Date', 'Symbol'], inplace=True)
df_weekly.tail()

,,Open,High,Low,Close,Volume
Date,Symbol,,,,,
2009-03-02 05:00:00,ZUMZ,7.51,8.59,7.45,7.45,317705
2009-03-03 05:00:00,ZUMZ,7.56,7.94,6.72,7.19,599659
2009-03-04 05:00:00,ZUMZ,7.33,7.64,7.08,7.17,256197
2009-03-05 05:00:00,ZUMZ,7.00,7.48,6.55,6.69,259952
2009-03-06 05:00:00,ZUMZ,6.76,6.95,6.27,6.43,240984


In [ ]:
df_weekly.to_csv('df_weekly.csv', index=False)

In [ ]:
def clean_dataset(df):
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
    return df[indices_to_keep].astype(np.float64)

In [ ]:
# #sklearn imports
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler

In [ ]:
# #https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html
# scaler = StandardScaler()

# features_std = scaler.fit_transform(df)

# feature_pca = PCA(n_components=4) 
# feature_pca.fit(df)

# print(feature_pca.explained_variance_ratio_)

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# corr = features.corr()

# #corr matrix with heatmap
# plt.figure(figsize=(6,5))
# sns.heatmap(corr, annot=False, cmap=plt.cm.Blues)
# # plt.savefig('feature_correlations.png')